In [1]:
import tensorflow as tf
print(f"Tensorflow version : {tf.__version__}")

Tensorflow version : 2.2.0-rc1


In [2]:
from tensorflow.python.ops import sparse_ops
import tensorflow.keras.backend as K
import pandas as pd
# import matplotlib.pyplot as plt
import os
assert tf.__version__=='2.2.0-rc1'
# assert tf.test.is_gpu_available()
K.clear_session()
%load_ext tensorboard
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from ipynb.fs.full.clustering import make_clusters, extract_features, tf_serialize_example

Tensorflow version : 2.2.0-rc1


## Parser

In [109]:
feature_description = {
    'eventId': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
    'image': tf.io.FixedLenFeature([3], tf.string, default_value=["",]*3)
}

feature_shape = {
    'eventId': tf.TensorShape([1,]),
    'image': tf.TensorShape([568, 568, 2])
}

def parser_fn(proto):
    serialized = tf.io.parse_single_example(proto, feature_description)
    deserialized = {k: (tf.sparse.to_dense(sparse_ops.deserialize_sparse(v, K.floatx()))
                        if k != 'eventId' else v) for k,v in serialized.items()}
    [deserialized[k].set_shape(feature_shape[k]) for k in deserialized.keys()]
    x = deserialized['image']
    x = tf.expand_dims(x, axis=0)
    x = K.pool2d(x, pool_size=(2, 2), strides=(2, 2), pool_mode='avg')
    # sum instead of avg
    x = 4.*x
    deserialized['image'] = tf.squeeze(x)
    deserialized['S_image'] = deserialized['image'][:,:,0]
    deserialized['C_image'] = deserialized['image'][:,:,1]
    
    # set EM633
    deserialized['tseed'] = tf.constant(6., tf.float32)
    deserialized['tneighbour'] = tf.constant(3., tf.float32)
    deserialized['tcell'] = tf.constant(3., tf.float32)
    deserialized['tenergy'] = tf.constant(1000., tf.float32)
    deserialized['tlocmax'] = tf.constant(500., tf.float32)
    deserialized['tnum'] = tf.constant(3, tf.int32)
    
    return deserialized

In [110]:
DATA_DIR = ''
BATCH_SIZE = 16
BUFFER_SIZE = 10

filename = [os.path.join(DATA_DIR, 'B4.tfrecord')]
dataset = tf.data.TFRecordDataset(filename, compression_type='GZIP', buffer_size=BUFFER_SIZE)
dataset = dataset.map(parser_fn)
clusters_dataset = dataset.take(10).map(make_clusters)
features_dataset = clusters_dataset.map(extract_features)
serialized_features_dataset = features_dataset.map(tf_serialize_example)

In [111]:
filename = 'test123.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

In [112]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "C_comi"
    value {
      float_list {
        value: 126.13424682617188
      }
    }
  }
  feature {
    key: "C_comj"
    value {
      float_list {
        value: 150.63902282714844
      }
    }
  }
  feature {
    key: "C_hot"
    value {
      float_list {
        value: 0.2284875363111496
      }
    }
  }
  feature {
    key: "C_rad_mean"
    value {
      float_list {
        value: 2.0498204231262207
      }
    }
  }
  feature {
    key: "C_sum"
    value {
      float_list {
        value: 30114.67578125
      }
    }
  }
  feature {
    key: "S_comi"
    value {
      float_list {
        value: 126.0400619506836
      }
    }
  }
  feature {
    key: "S_comj"
    value {
      float_list {
        value: 150.54531860351562
      }
    }
  }
  feature {
    key: "S_hot"
    value {
      float_list {
        value: 0.11042886972427368
      }
    }
  }
  feature {
    key: "S_rad_mean"
    value {
      float_list {
        value: 2.837070

In [113]:
for output in clusters_dataset.take(10):
    plot_output(output)

Info in <TCanvas::Print>: png file cluster0.png has been created
Info in <TCanvas::Print>: png file cluster1.png has been created
Info in <TCanvas::Print>: png file cluster2.png has been created
Info in <TCanvas::Print>: png file cluster3.png has been created
Info in <TCanvas::Print>: png file cluster4.png has been created
Info in <TCanvas::Print>: png file cluster5.png has been created
Info in <TCanvas::Print>: png file cluster6.png has been created
Info in <TCanvas::Print>: png file cluster7.png has been created
Info in <TCanvas::Print>: png file cluster8.png has been created
Info in <TCanvas::Print>: png file cluster9.png has been created


In [114]:
import ROOT
from array import array

def plot_output(output):
    h = ROOT.TH2F("","", 284, 0., 284., 284, 0., 284.)
    c = ROOT.TCanvas("c1","c1", 800, 400)
    c.Divide(2)
    ROOT.gStyle.SetOptStat(0)
    palette = [ROOT.kRed, ROOT.kGreen, ROOT.kYellow, ROOT.kBlue, ROOT.kRed, ROOT.kGreen, ROOT.kYellow, ROOT.kBlue]
    for l,X in enumerate(['S', 'C']):
        c.cd(l+1)
        ROOT.gPad.SetLeftMargin(0.05)
        ROOT.gPad.SetBottomMargin(0.05)
        ROOT.gPad.SetRightMargin(0.15)
        ROOT.gPad.SetTopMargin(0.15)
        X_image = output[X+'_image']
        indices = tf.where(X_image)
        values = tf.reshape(tf.gather_nd(X_image, indices), [-1,1])
        X_image = tf.concat([tf.cast(indices, values.dtype), values], axis=1)

        htot = h.Clone()
        for i,j,x in X_image:
            htot.Fill(j,i,x)
        htot.DrawCopy("COLZ")

        X_cluster = output[X+'_cluster'].numpy()
        if X_cluster.size != 0:
            hk = []
            u = np.unique(X_cluster[:,0], axis=0).astype(int)
            arr = array('d',[0.5])
            for k in range(len(u)):
                for i,j,x in X_cluster[X_cluster[...,0]==u[k]][:,1:]:
                    hk.append(h.Clone())
                    hk[k].Fill(j,i,x)

            for k in range(len(u)):
                hk[k].SetLineColor(palette[k])
                hk[k].SetLineWidth(1)
                hk[k].SetContour(1, arr)
                hk[k].DrawCopy("cont3 list same")

    c.SaveAs("cluster"+str(output['eventId'][0].numpy())+".png")